## Problem: Maximize Profit from these members. 
### Where to go from here? - questions? observations? directions?
#### • emails.tsv: a tab-separated file of all emails sent to a subset of our members from the JobsRadar brand in September
Column1: email_id
Column2: email_send_time (EST)
Column3: email_type
	T plus 1 is a campaign sent 1 day after a member joins
	T plus N are campaigns sent N days after a member joins
	Transactional Forgot Password Email
	Transactional JR Welcome Email
Column4: email_variant
	First Part:
		fixed_keyword_cloud = keyword cloud email (See samples)
		job_alert = custom job listings (see samples)
	2nd part:
		tplusN helps with time since joined (joined N days ago)
		age22+ goes to people age 22+ (includes anyone 22-999, meaning everyone except under 22),
		age35+ likewise
		1opened goes to people that opened at least once before.
Column5: member_id

job_alert_s1_v1 is a specific “version” of a “job alert” type of email we send.
tplus2 means this message was sent to members that joined our site 2 days ago
tplus201 means this message was sent to members that joined our site 201 days ago
age22+ means it was sent to people that were at least 22 years old.
1opened means it was sent to people that had opened at least 1 message from us in the past (“Openers”)

#### • email_responses.tsv: a tab-separated file of open, click, and unsubscribe events that resulted from those emails
#### • members.tsv: a tab-separated file of information about the members to whom those emails were sent
#### • *.png: sample images of each of the email variants sent in this set (as identified by either the "variant" or "campaign" column in emails.tsv)
#### • *.eml: Outlook export files of same

#### A) we make 0.12 per click event 
#### B) it costs us 0.40 for every 1,000 emails sent.
   

In [1]:
# Import some useful python modules
import matplotlib as plt
import seaborn as sns
%matplotlib inline

import numpy as np
import pandas as pd
import os
import datetime
import sqlite3
from sqlalchemy import create_engine, text
import nltk

def sqldf(query, sql_engine):
    query_text = text(query)
    results = pd.read_sql_query(query_text, con=sql_engine)
    return results

def sqlraw():
    conn = sqlite3.connect('jobcase.db')
    return conn

In [48]:
nltk.download()

In [2]:
# change working directory into folder with data, view what is available
os.chdir(os.getcwd() + '\data')

In [3]:
conn = sqlite3.connect('jobcase.db')
engine = create_engine('sqlite:///C:\\Users\\Dan\\1) Python Notebooks\\Jobcase\\data\\jobcase.db')
engine.raw_connection().connection.text_factory = str
conn.close()
connection = engine.connect()

In [3]:
# Parse data into Pandas DataFrames
# emails
emails = pd.read_table('emails.tsv', header=None)
emails.columns = ['email_id','timestamp','email_type','email_variant','member_id']
emails.email_variant = emails.email_variant.apply(lambda x: str(x).replace('\\N', 'NONE')) 
emails['email_variant_first_part'] = emails['email_variant'].apply(lambda x: x.split(':')[0] if not pd.isnull(x) else np.nan)
emails['email_variant_second_part'] = emails['email_variant'].apply(lambda x: x.split(':')[1] if not pd.isnull(x) and len(x.split(':')) > 1 else np.nan)
emails.member_id = emails.member_id.astype(str)

# email responses
email_responses = pd.read_table('email_responses.tsv', header=0)

In [4]:
email_responses['action'].value_counts()

open     1099755
click     436201
unsub      22653
dtype: int64

In [75]:
# Calculating the profit based on the amounts identified ($0.4/1000 and $0.12/email)
total_cost = len(emails.email_id) * 0.4 / 1000
gross_revenue = email_responses['action'].value_counts().click * 0.12
profit = gross_revenue - total_cost
print gross_revenue
print total_cost
print profit 

52344.12
3843.1236
48500.9964


Based on this dataset, we can see that we have sent 9,607,809 emails. Sending these emails cost \$0.4/1000 emails for a total cost of \$3,843.12. Looking at the email responses we can see that there are 436,201 'click' events. Each click event earns Jobcase earns \$0.12/click. This means that based on this dataset we have made \$52,344.12 in revenue. Our profit on this dataset is then \$48,501.

In [4]:
# There are some issues with the members file (varying apparent row lengths, likely due to a separator issue) 
# so we will read that directly and try to identify where the issues are. 
# After the data is in a usable form and then parse the data into a dataframe manually
members_file = open('members.tsv','r')
members_list = []
members_list_raw = []
for line in members_file.readlines():
    members_list_raw.append(line)
    members_list.append(line.replace('\n','').split('\t'))

#print members_list[0]
#members_list_raw[205009]

# From investigating the above rows, we can see that the issue stems from additional tabs in the value fields
# we therefore need to replace the invalid dditional tab values so the data can be formatted correctly
clean_members_list = [
    [unicode(col.decode("ascii","ignore")) 
        for col in member.replace('\n','').replace('\\\t','').replace('\N','').replace("'","").replace('+','').split('\t')
    ] for member in members_list_raw
]

members = pd.DataFrame(data=clean_members_list[1:], 
                       columns=clean_members_list[0])

In [5]:
emails_and_clicks = pd.merge(emails, email_responses[email_responses.action=='click'], how='left', on='email_id', suffixes=['_1', '_2'])
emails_and_resp = pd.merge(emails_and_clicks
                           , pd.DataFrame(data=email_responses.email_id.unique(),columns=['email_id'])
                           , how='inner', on='email_id', suffixes=['_1', '_2'])
emails_and_resp['action'] = emails_and_resp['action'].fillna('na')

In [63]:
emails_and_resp.email_id.count()

963131

In [6]:
emails_and_members = pd.merge(emails_and_resp, members, how='inner', on='member_id', suffixes=['_1', '_2'])
emails_and_members.keyword = emails_and_members.keyword.fillna('').apply(lambda x: x.lower().replace('jobs','').replace('job','').replace('www','').replace('.com',''))

In [75]:
#emails.to_sql('emails', engine)
#email_responses.to_sql('email_responses', engine)
#members.to_sql('members', engine, if_exists='replace')
#emails_and_members.to_sql('emails_and_members', engine)

# Normalizing the Features

In [7]:
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize

stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

vectorizer = TfidfVectorizer(max_df=0.5, max_features=30, min_df=2, stop_words='english',tokenizer=tokenize)
vX = vectorizer.fit_transform(emails_and_members.keyword)

In [8]:
emails_and_members['variant_vals'] = emails_and_members.email_variant.fillna('none').apply(lambda x: x.lower().replace('_',' ').replace(':',' ').replace('+',' '))

In [9]:
stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

vectorizer_variant_vals = TfidfVectorizer(max_df=0.5, max_features=1000, min_df=2, stop_words='english', tokenizer=tokenize)
vX_variants = vectorizer_variant_vals.fit_transform(emails_and_members.variant_vals)

In [35]:
emails_and_members.head()

,email_id,timestamp_1,email_type,email_variant,member_id,email_variant_first_part,email_variant_second_part,timestamp_2,action,date,email_domain,first_name,city,state,zip,degree_level,hs_or_ged_year,pcp_score,keyword,variant_vals
0,205570076,2012-09-01 00:10:08,Transactional JR Welcome Email,account_login_info_s2_v1,14802260,account_login_info_s2_v1,NaN,2012-09-01 00:23:49,click,2012-09-01 01:02:03,yahoo.com,michael,SPARTA,TN,38583,Some HS,2013,0.10344,kroger,account login info s2 v1
1,205570123,2012-09-01 00:11:07,Transactional Forgot Password Email,nan,8450299,nan,NaN,2012-09-01 00:12:57,click,2011-07-13 21:57:47,hotmail.com,Maria,Chula vista,CA,91911,Associate,1979,0.320591,costco,nan
2,205570320,2012-09-01 00:30:08,Transactional JR Welcome Email,account_login_info_s2_v1,14802278,account_login_info_s2_v1,NaN,2012-09-01 00:31:09,click,2012-09-01 01:24:50,yahoo.com,Frank,FORT LAUDERDALE,FL,33301,Some College,1981,0.174869,fedex,account login info s2 v1
3,206014887,2012-09-02 04:00:08,T plus N,fixed_keyword_cloud_s1_v1:tplus1_age22+,14802278,fixed_keyword_cloud_s1_v1,tplus1_age22+,2012-09-02 15:35:01,click,2012-09-01 01:24:50,yahoo.com,Frank,FORT LAUDERDALE,FL,33301,Some College,1981,0.174869,fedex,fixed keyword cloud s1 v1 tplus1 age22
4,206384628,2012-09-03 04:00:07,T plus N,fixed_keyword_cloud_s1_v1:tplus2,14802278,fixed_keyword_cloud_s1_v1,tplus2,2012-09-03 05:18:37,click,2012-09-01 01:24:50,yahoo.com,Frank,FORT LAUDERDALE,FL,33301,Some College,1981,0.174869,fedex,fixed keyword cloud s1 v1 tplus2


In [33]:
emails_and_members.head()

,email_id,timestamp_1,email_type,email_variant,member_id,email_variant_first_part,email_variant_second_part,timestamp_2,action,date,email_domain,first_name,city,state,zip,degree_level,hs_or_ged_year,pcp_score,keyword,variant_vals
0,205570076,2012-09-01 00:10:08,Transactional JR Welcome Email,account_login_info_s2_v1,14802260,account_login_info_s2_v1,NaN,2012-09-01 00:23:49,click,2012-09-01 01:02:03,yahoo.com,michael,SPARTA,TN,38583,Some HS,2013,0.10344,kroger,account login info s2 v1
1,205570123,2012-09-01 00:11:07,Transactional Forgot Password Email,nan,8450299,nan,NaN,2012-09-01 00:12:57,click,2011-07-13 21:57:47,hotmail.com,Maria,Chula vista,CA,91911,Associate,1979,0.320591,costco,nan
2,205570320,2012-09-01 00:30:08,Transactional JR Welcome Email,account_login_info_s2_v1,14802278,account_login_info_s2_v1,NaN,2012-09-01 00:31:09,click,2012-09-01 01:24:50,yahoo.com,Frank,FORT LAUDERDALE,FL,33301,Some College,1981,0.174869,fedex,account login info s2 v1
3,206014887,2012-09-02 04:00:08,T plus N,fixed_keyword_cloud_s1_v1:tplus1_age22+,14802278,fixed_keyword_cloud_s1_v1,tplus1_age22+,2012-09-02 15:35:01,click,2012-09-01 01:24:50,yahoo.com,Frank,FORT LAUDERDALE,FL,33301,Some College,1981,0.174869,fedex,fixed keyword cloud s1 v1 tplus1 age22
4,206384628,2012-09-03 04:00:07,T plus N,fixed_keyword_cloud_s1_v1:tplus2,14802278,fixed_keyword_cloud_s1_v1,tplus2,2012-09-03 05:18:37,click,2012-09-01 01:24:50,yahoo.com,Frank,FORT LAUDERDALE,FL,33301,Some College,1981,0.174869,fedex,fixed keyword cloud s1 v1 tplus2


In [10]:
# creating dummy values for the categorical variables
features = pd.concat(objs=[pd.get_dummies(emails_and_members.email_type), 
                           pd.get_dummies(emails_and_members.email_variant),
                           #pd.get_dummies(emails_and_members.degree_level),
                           pd.get_dummies(emails_and_members.hs_or_ged_year),
                           #emails_and_members.pcp_score,
                           #pd.DataFrame(data=vX.toarray(), columns = vectorizer.get_feature_names()),
                           pd.DataFrame(data=vX_variants.toarray(), columns = vectorizer_variant_vals.get_feature_names())
                          ], axis=1)
features.head()
target = pd.get_dummies(emails_and_members.action).click



# Now building the model


In [29]:
def model_performance(model, features, target):
    total_pred = model.predict(features)
    prod_predictions = pd.DataFrame(data=total_pred, columns=['predicted'])
    prod_predictions['actual'] = target
    print '\nprediction counts\n', prod_predictions[prod_predictions.predicted == 1].actual.value_counts()
    print '\ntotal predicted ', sum(prod_predictions[prod_predictions.predicted == 1].count()) 
    revenue = len(prod_predictions[(prod_predictions.predicted == 1) & (prod_predictions.actual == 1)]) * .12
    cost =len(prod_predictions[(prod_predictions.predicted == 1)]) * .4/1000
    print 'revenue = ', revenue
    print 'cost = ', cost
    print 'profit = ', revenue-cost

In [12]:
X = features.as_matrix().astype(int).astype(float)
y = target
# This is important
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

print "Feature space holds %d observations and %d features" % X.shape
print "Unique target labels:", np.unique(y)

Feature space holds 932706 observations and 317 features
Unique target labels: [ 0.  1.]


In [42]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier, RidgeClassifierCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.grid_search import GridSearchCV

from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

#X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
#clf = LogisticRegression(class_weight='auto')
#GaussianNB
clf = SVC(kernel='linear')
clf.fit(X_train, y_train.ravel())
predictions = clf.predict(X_test)
print str(clf).split('(')[0], ' Classifier'

print classification_report(y_test, predictions, target_names=['no click','click'])
print confusion_matrix(y_test, predictions)
print '\n-------------------------\n'
model_performance(model=clf, features=X, target=y)

In [31]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features_train = rbf_feature.fit_transform(X_train)
X_features_test = rbf_feature.fit_transform(X_test)

clf = GaussianNB()
clf.fit(X_features_train, y_train.ravel())
predictions = clf.predict(X_features_test)
print str(clf).split('(')[0], ' Classifier'

print classification_report(y_test, predictions, target_names=['no click','click'])
print confusion_matrix(y_test, predictions)
print '\n-------------------------\n'
model_performance(model=clf, features=rbf_feature.fit_transform(X), target=y)

GaussianNB  Classifier
             precision    recall  f1-score   support

   no click       0.55      0.86      0.67    126078
      click       0.52      0.18      0.27    107099

avg / total       0.54      0.55      0.49    233177

[[107898  18180]
 [ 87739  19360]]

-------------------------


prediction counts
1    77522
0    73829
dtype: int64

total predicted  302702
revenue =  9302.64
cost =  60.5404
profit =  9242.0996


In [120]:
clf = SGDClassifier()
sgd_parameters = {'loss' : ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                  'penalty' : ['none', 'l2', 'l1', 'elasticnet'],
                  'alpha': [0.0001, 0.1, 1, 5, 10],
                  'class_weight': ['auto']
             }
gs = GridSearchCV(clf, sgd_parameters)
#clf = LogisticRegression(class_weight='auto')
gs.fit(X_train, y_train.ravel())
predictions = gs.predict(X_test)
print str(clf).split('(')[0], ' Classifier'

print classification_report(y_test, predictions, target_names=['no click','click'])
print confusion_matrix(y_test, predictions)
print '\n-------------------------\n'
#model_performance(gs)

SGDClassifier  Classifier
             precision    recall  f1-score   support

   no click       0.64      0.69      0.67    125841
      click       0.60      0.55      0.58    107336

avg / total       0.62      0.63      0.62    233177

[[86606 39235]
 [47879 59457]]

-------------------------



In [122]:
gs.get_params()

{'cv': None,
 'error_score': 'raise',
 'estimator': SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
        eta0=0.0, fit_intercept=True, l1_ratio=0.15,
        learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
        penalty='l2', power_t=0.5, random_state=None, shuffle=True,
        verbose=0, warm_start=False),
 'estimator__alpha': 0.0001,
 'estimator__average': False,
 'estimator__class_weight': None,
 'estimator__epsilon': 0.1,
 'estimator__eta0': 0.0,
 'estimator__fit_intercept': True,
 'estimator__l1_ratio': 0.15,
 'estimator__learning_rate': 'optimal',
 'estimator__loss': 'hinge',
 'estimator__n_iter': 5,
 'estimator__n_jobs': 1,
 'estimator__penalty': 'l2',
 'estimator__power_t': 0.5,
 'estimator__random_state': None,
 'estimator__shuffle': True,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'fit_params': {},
 'iid': True,
 'loss_func': None,
 'n_jobs': 1,
 'param_grid': {'alpha': [0.0001, 0.1, 1, 5, 10],
  'class_weight': ['a

In [ ]:
clf = LogisticRegression()
sgd_parameters = {'penalty' : ['l2', 'l1'],
                  'solver': ['newton-cg', 'lbfgs', 'liblinear'],
                  'C': [1.0, 0.5, 0.1],
                  'class_weight': ['auto']
             }
gs = GridSearchCV(clf, sgd_parameters)
gs.fit(X_train, y_train.ravel())
predictions = gs.predict(X_test)
print str(clf).split('(')[0], ' Classifier'

print classification_report(y_test, predictions, target_names=['no click','click'])
print confusion_matrix(y_test, predictions)
print '\n-------------------------\n'
#model_performance(gs)

In [98]:
clf = SGDClassifier(class_weight='auto')
#clf = LogisticRegression(class_weight='auto')
clf.fit(X_train, y_train.ravel())
predictions = clf.predict(X_test)
print str(clf).split('(')[0], ' Classifier'

print classification_report(y_test, predictions, target_names=['no click','click'])
print confusion_matrix(y_test, predictions)
print '\n-------------------------\n'
model_performance()

SGDClassifier  Classifier
             precision    recall  f1-score   support

        0.0       0.64      0.54      0.59    125823
        1.0       0.55      0.65      0.59    107354

avg / total       0.60      0.59      0.59    233177

[[67753 58070]
 [37466 69888]]

-------------------------


prediction counts
1    279572
0    232846
dtype: int64

total predicted  1024836
revenue =  33548.64
cost =  204.9672
profit =  33343.6728


In [97]:
model_performance()


prediction counts
1    277164
0    197314
dtype: int64

total predicted  948956
revenue =  33259.68
cost =  189.7912
profit =  33069.8888


In [81]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print 'Decision Tree Classifier'
print classification_report(y_test, predictions)

Decision Tree Classifier
             precision    recall  f1-score   support

        0.0       0.64      0.62      0.63    126357
        1.0       0.57      0.60      0.58    106820

avg / total       0.61      0.61      0.61    233177



In [82]:
clf = RandomForestClassifier(n_estimators=10)
clf.fit(X_train, y_train.ravel())
predictions = clf.predict(X_test)
print 'RandomForest Classifier'
print classification_report(y_test, predictions)

RandomForest Classifier
             precision    recall  f1-score   support

        0.0       0.65      0.61      0.63    126357
        1.0       0.57      0.60      0.59    106820

avg / total       0.61      0.61      0.61    233177



In [83]:
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train.ravel())
predictions = clf.predict(X_test)
print 'Gradient Boosting Classifier'
print classification_report(y_test, predictions)

Gradient Boosting Classifier
             precision    recall  f1-score   support

        0.0       0.66      0.54      0.60    126357
        1.0       0.55      0.68      0.61    106820

avg / total       0.61      0.60      0.60    233177



In [16]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train.ravel())
predictions = clf.predict(X_test)
print 'Gradient Boosting Classifier'
print classification_report(y_test, predictions)

Gradient Boosting Classifier
             precision    recall  f1-score   support

        0.0       0.91      0.00      0.00    258449
        1.0       0.29      1.00      0.46    107894

avg / total       0.73      0.29      0.13    366343



In [18]:
clf = ExtraTreesClassifier()
clf.fit(X_train, y_train.ravel())
predictions = clf.predict(X_test)
print 'Extra Trees Classifier'
print classification_report(y_test, predictions)

Extra Trees Classifier
             precision    recall  f1-score   support

        0.0       0.71      1.00      0.83    258449
        1.0       1.00      0.00      0.00    107894

avg / total       0.79      0.71      0.58    366343



In [17]:
clf = LogisticRegression()
clf.fit(X_train, y_train.ravel())
predictions = clf.predict(X_test)
print 'Logistic Regression Classifier'
print classification_report(y_test, predictions)

Logistic Regression Classifier
             precision    recall  f1-score   support

        0.0       0.71      1.00      0.83    258449
        1.0       1.00      0.00      0.00    107894

avg / total       0.79      0.71      0.58    366343



In [ ]:
# instantiate the model (using the value K=5)
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train.ravel())
predictions = knn.predict(X_test)
print 'K Nearest Neighbor Regression Classifier'
print classification_report(y_test, predictions)

In [ ]:
clf = SVC()
clf.fit(X_train, y_train.ravel())
predictions = clf.predict(X_test)
print 'Support Vector Machines Classifier'
print classification_report(y_test, predictions)

In [14]:
clf = LinearSVC()
clf.fit(X_train, y_train.ravel())
predictions = clf.predict(X_test)
print 'Support Vector Machines Classifier'
print classification_report(y_test, predictions)

Support Vector Machines Classifier
             precision    recall  f1-score   support

        0.0       0.71      1.00      0.83    258949
        1.0       0.67      0.00      0.00    107394

avg / total       0.70      0.71      0.59    366343



In [91]:
clf = SGDClassifier()
clf.fit(X_train, y_train.ravel())
predictions = clf.predict(X_test)
print 'SGD Classifier'
print classification_report(y_test, predictions)

SGD Classifier
             precision    recall  f1-score   support

        0.0       0.96      1.00      0.98    224300
        1.0       0.00      0.00      0.00      8877

avg / total       0.93      0.96      0.94    233177



In [ ]:
from sklearn.cross_validation import KFold

def run_cv(X,y,clf_class,**kwargs):
    # Construct a kfolds object
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_pred = y.copy()
    # Iterate through folds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        # Initialize a classifier with key word arguments
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

def accuracy(y_true,y_pred):
    # NumPy interprets True and False as 1. and 0.
    return np.mean(y_true == y_pred)

print "Random forest:"
print "%.3f" % accuracy(y, run_cv(X,y,GradientBoostingClassifier))


In [10]:
from sklearn.cross_validation import KFold

def run_cv(X,y,clf_class,**kwargs):
    # Construct a kfolds object
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_pred = y.copy()
    
    # Iterate through folds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        # Initialize a classifier with key word arguments
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LR

def accuracy(y_true,y_pred):
    # NumPy interprets True and False as 1. and 0.
    return np.mean(y_true == y_pred)

#print "Support vector machines:"
#print "%.3f" % accuracy(y, run_cv(X,y,SVC))
print "Random forest:"
print "%.3f" % accuracy(y, run_cv(X,y,RF))
print "K-nearest-neighbors:"
print "%.3f" % accuracy(y, run_cv(X,y,KNN))
print "Logistic Regression:"
print "%.3f" % accuracy(y, run_cv(X,y,LR))

In [13]:
def run_prob_cv(X, y, clf_class, **kwargs):
    kf = KFold(len(y), n_folds=5, shuffle=True)
    y_prob = np.zeros((len(y),2))
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        # Predict probabilities, not classes
        y_prob[test_index] = clf.predict_proba(X_test)
    return y_prob

In [ ]:
# Use 10 estimators so predictions are all multiples of 0.1
pred_prob = run_prob_cv(X, y, RF, n_estimators=10)
pred_churn = pred_prob[:,1]
is_churn = y == 1

# Number of times a predicted probability is assigned to an observation
counts = pd.value_counts(pred_churn)

# calculate true probabilities
true_prob = {}
for prob in counts.index:
    true_prob[prob] = np.mean(is_churn[pred_churn == prob])
    true_prob = pd.Series(true_prob)

# pandas-fu
counts = pd.concat([counts,true_prob], axis=1).reset_index()
counts.columns = ['pred_prob', 'count', 'true_prob']
counts

In [ ]:
# Now running the predictions based on the model we created
model = 
model.fit(X)
predictions = model.predict(X)
predictions.value_counts()


In [35]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import brier_score_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cross_validation import train_test_split

sample_weight = np.random.RandomState(42).rand(y.shape[0])

# split train, test for calibration
X_train, X_test, y_train, y_test, sw_train, sw_test = \
    train_test_split(X, y, sample_weight, test_size=0.9, random_state=42)

# Gaussian Naive-Bayes with no calibration
clf = GaussianNB()
clf.fit(X_train, y_train)  # GaussianNB itself does not support sample-weights
prob_pos_clf = clf.predict_proba(X_test)[:, 1]

# Gaussian Naive-Bayes with isotonic calibration
clf_isotonic = CalibratedClassifierCV(clf, cv=2, method='isotonic')
clf_isotonic.fit(X_train, y_train, sw_train)
#prob_pos_isotonic = clf_isotonic.predict_proba(X_test)[:, 1]

# Gaussian Naive-Bayes with sigmoid calibration
clf_sigmoid = CalibratedClassifierCV(clf, cv=2, method='sigmoid')
clf_sigmoid.fit(X_train, y_train, sw_train)
#prob_pos_sigmoid = clf_sigmoid.predict_proba(X_test)[:, 1]

print("Brier scores: (the smaller the better)")
#clf_score = brier_score_loss(y_test, prob_pos_clf, sw_test)
#print("No calibration: %1.3f" % clf_score)

#clf_isotonic_score = brier_score_loss(y_test, prob_pos_isotonic, sw_test)
#print("With isotonic calibration: %1.3f" % clf_isotonic_score)

#clf_sigmoid_score = brier_score_loss(y_test, prob_pos_sigmoid, sw_test)
#print("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

Brier scores: (the smaller the better)


In [38]:
predictions = clf_sigmoid.predict(X_test)
print str(clf_sigmoid).split('(')[0], ' Classifier'

print classification_report(y_test, predictions, target_names=['no click','click'])
print confusion_matrix(y_test, predictions)
print '\n-------------------------\n'
model_performance(model=clf_sigmoid, features=X, target=y)

CalibratedClassifierCV  Classifier
             precision    recall  f1-score   support

   no click       0.71      0.35      0.47    453585
      click       0.52      0.83      0.64    385851

avg / total       0.62      0.57      0.55    839436

[[158828 294757]
 [ 64895 320956]]

-------------------------


prediction counts
1    356822
0    327136
dtype: int64

total predicted  1367916
revenue =  42818.64
cost =  273.5832
profit =  42545.0568


In [39]:
predictions = clf_isotonic.predict(X_test)
print str(clf_isotonic).split('(')[0], ' Classifier'

print classification_report(y_test, predictions, target_names=['no click','click'])
print confusion_matrix(y_test, predictions)
print '\n-------------------------\n'
model_performance(model=clf_isotonic, features=X, target=y)

CalibratedClassifierCV  Classifier
             precision    recall  f1-score   support

   no click       0.70      0.37      0.49    453585
      click       0.53      0.81      0.64    385851

avg / total       0.62      0.58      0.56    839436

[[169788 283797]
 [ 71518 314333]]

-------------------------


prediction counts
1    349483
0    314965
dtype: int64

total predicted  1328896
revenue =  41937.96
cost =  265.7792
profit =  41672.1808


In [40]:
predictions = clf.predict(X_test)
print str(clf).split('(')[0], ' Classifier'

print classification_report(y_test, predictions, target_names=['no click','click'])
print confusion_matrix(y_test, predictions)
print '\n-------------------------\n'
model_performance(model=clf, features=X, target=y)

GaussianNB  Classifier
             precision    recall  f1-score   support

   no click       0.71      0.36      0.47    453585
      click       0.52      0.83      0.64    385851

avg / total       0.62      0.57      0.55    839436

[[162003 291582]
 [ 66873 318978]]

-------------------------


prediction counts
1    354643
0    323596
dtype: int64

total predicted  1356478
revenue =  42557.16
cost =  271.2956
profit =  42285.8644
